# IMPORTANT: use a python 2 kernel for cvxpy to work!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cvxpy import *
%matplotlib inline
import pandas as pd

# Part 1: Data Import

In [5]:
# Import and clean data:

node_data = pd.read_excel('Robust_Optimization_Data.xlsx', sheet_name="Node-Data")
node_data = node_data.drop(node_data.index[:3])
node_data = node_data.drop(node_data.index[-1])
node_data.columns = ['nodes', 'active power', 'reactive power', 'generating power', 'generation cost']
node_data = node_data.astype('float')
node_data = node_data.set_index('nodes')
node_data

,active power,reactive power,generating power,generation cost
nodes,,,,
0.0,0.000,0.000,5.0,100.0
1.0,0.200,0.116,0.0,0.0
2.0,0.000,0.000,0.0,0.0
3.0,0.400,0.290,3.0,150.0
4.0,0.170,0.125,0.0,0.0
5.0,0.230,0.132,0.0,0.0
6.0,1.155,0.660,0.0,0.0
7.0,0.000,0.000,0.0,0.0
8.0,0.170,0.151,0.0,0.0


# Part 2: Parameters

In [76]:
# Node Data
# l_j^P: Active power consumption [MW]
l_P = np.array([0., 0.200, 0., 0.400, 0.170, 0.230, 1.155, 0., 0.170, 0.843, 0., 0.170, 0.128])

# l_j^Q: Reactive power consumption [MVAr]
l_Q = np.array([0., 0.116, 0., 0.290, 0.125, 0.132, 0.660, 0., 0.151, 0.462, 0., 0.080, 0.086])

# l_j^S: Apparent power consumption [MVA]
l_S = np.sqrt(l_P**2 + l_Q**2)

# s_j,max: Maximum generating power [MW]
s_max = np.array([5.000, 0., 0., 3.000, 0., 0., 0., 0., 0., 3.000, 0., 0., 0.])

# c_j: Marginal generation cost [USD/MW]
c = np.array([100., 0., 0., 150., 0., 0., 0., 0., 0., 50., 0., 0., 0.])

# Line Data
# r_ij: Resistance [p.u.]
r = np.array([[0, 0.007547918, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0.0041, 0, 0.007239685, 0, 0.007547918, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0.004343811, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0.003773959, 0, 0, 0.004322245, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00434686, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.004343157, 0.01169764],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

# x_ij: Reactance [p.u.]
x = np.array([[0, 0.022173236, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0.0064, 0, 0.007336076, 0, 0.022173236, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0.004401645, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0.011086618, 0, 0, 0.004433667, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.002430473, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.004402952, 0.004490848],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

# I_max_ij: Maximal line current [p.u.]
I_max = np.array([[0, 3.0441, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1.4178, 0, 0.9591, 0, 3.0441, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 3.1275, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0.9591, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 3.0441, 3.1275, 0, 0.9591, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1.37193, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9591, 1.2927],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

# A_ij: Adjacency matrix; A_ij = 1 if i is parent of j
A = np.array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


# Network Data
# node indices
j_idx = np.arange(13)

# \rho(j): Parent node of node j
rho = np.array([0, 0, 1, 2, 1, 4, 1, 6, 6, 8, 6, 10, 10])

### These values can be adjusted 

In [95]:
# V_min, V_max: Minimum and maximum nodal voltages [V]
v_min = 0.95
v_max = 1.05

# Part 3: Optimization

## Reliable Renewable Generation

In [89]:
# Define optimization variables
P = Variable((13,13))
Q = Variable((13,13))
L = Variable((13,13))
p = Variable(13)
q = Variable(13)
s = Variable(13)
V = Variable(13)

# Define objective function
objective = Minimize(c.T*s)

# Define constraints
# Apparent Power Limits
constraints = [s <= s_max]


# Nodal voltage limits
constraints += [v_min**2 <= V, V <= v_max**2]


# Squared line current limits
constraints += [L[0,0] <= I_max**2]


# Boundary condition for power line flows
constraints += [P[0,0] == 0, Q[0,0] == 0]


# Boundary condition for squared line current
constraints += [L[0] == 0]


# Fix node 0 voltage to be 1 "per unit" (p.u.)
constraints += [V[0] == 1]


# Loop over each node
for jj in j_idx:
    
    # Parent node, i = \rho(j)
    ii = rho[jj]
    
    # Line Power Flows
    constraints += [P[ii,jj] == l_P[jj] - p[jj] + r[ii,jj]*L[ii,jj] + A[jj,:]*P[jj,:].T,
                    Q[ii,jj] == l_Q[jj] - q[jj] + x[ii,jj]*L[ii,jj] + A[jj,:]*Q[jj,:].T]

    # Nodal voltage
    constraints += [V[jj] == V[ii] + (r[ii,jj]**2 + x[ii,jj]**2)*L[ii,jj] - 2*(r[ii,jj]*P[ii,jj] + x[ii,jj]*Q[ii,jj])]
    
    # Squared current magnitude on lines
    constraints += [L[ii,jj] >= quad_over_lin(vstack((P[ii,jj],Q[ii,jj])),V[jj])]
    
    # Compute apparent power from active & reactive power
    constraints += [norm(hstack((p[jj],q[jj]))) <= s[jj]]
    

# Define problem and solve
prob1 = Problem(objective, constraints)
prob1.solve()

# Output Results
print "Minimum Generating Cost : %4.2f"%(prob1.value),"USD"
print " "
print "Node 0 Nuclear Power: active = %1.3f"%(p[0].value), "MW | reactive = %1.3f"%(q[0].value), "MW | apparent = %1.3f"%(s[0].value),"MW || marginal savings = %3.0f"%(constraints[0].dual_value[0]), "USD/MW"
print "Node 3 FF Power: active = %1.3f"%(p[3].value), "MW | reactive = %1.3f"%(q[3].value), "MW | apparent = %1.3f"%(s[3].value),"MW || marginal savings = %3.0f"%(constraints[0].dual_value[3]), "USD/MW"
print "Node 9 Solar Power: active = %1.3f"%(p[9].value), "MW | reactive = %1.3f"%(q[9].value), "MW | apparent = %1.3f"%(s[9].value),"MW || marginal savings = %3.0f"%(constraints[0].dual_value[9]), "USD/MW"
print " "
print "Total active power   : %1.3f"%(np.sum(l_P)),"MW   demanded | %1.3f"%(np.sum(p.value)),"MW   generated"
print "Total reactive power : %1.3f"%(np.sum(l_Q)),"MVAr demanded | %1.3f"%(np.sum(q.value)),"MVAr generated"
print "Total apparent power : %1.3f"%(np.sum(l_S)),"MVA  demanded | %1.3f"%(np.sum(s.value)),"MVA  generated"
print " "
for jj in j_idx:
    print "Node %2.0f"%(jj), "Voltage : %1.3f"%((V[jj].value)**0.5), "p.u."

Minimum Generating Cost : 311.99 USD
 
Node 0 Nuclear Power: active = 0.000 MW | reactive = 0.000 MW | apparent = 0.000 MW || marginal savings =   0 USD/MW
Node 3 FF Power: active = 0.922 MW | reactive = 0.562 MW | apparent = 1.080 MW || marginal savings =   0 USD/MW
Node 9 Solar Power: active = 2.565 MW | reactive = 1.555 MW | apparent = 3.000 MW || marginal savings =  98 USD/MW
 
Total active power   : 3.466 MW   demanded | 3.488 MW   generated
Total reactive power : 2.102 MVAr demanded | 2.117 MVAr generated
Total apparent power : 4.063 MVA  demanded | 4.080 MVA  generated
 
Node  0 Voltage : 1.000 p.u.
Node  1 Voltage : 1.000 p.u.
Node  2 Voltage : 1.004 p.u.
Node  3 Voltage : 1.004 p.u.
Node  4 Voltage : 0.995 p.u.
Node  5 Voltage : 0.994 p.u.
Node  6 Voltage : 1.003 p.u.
Node  7 Voltage : 1.003 p.u.
Node  8 Voltage : 1.003 p.u.
Node  9 Voltage : 1.013 p.u.
Node 10 Voltage : 1.001 p.u.
Node 11 Voltage : 1.000 p.u.
Node 12 Voltage : 0.999 p.u.


## Uncertain Renewable Generation 

In [97]:
# Second order cone constraint parameters
a_bar = np.array([[1.], [-1.25], [-1.25]])
# E = np.diag([0, 0.25, 0.25])

E = np.array([[1, 0, 0], [0, 0.25, 0], [0, 0, 0.25]])

# Define optimization vars
P = Variable((13,13))
Q = Variable((13,13))
L = Variable((13,13))
p = Variable(13)
q = Variable(13)
s = Variable(13)
V = Variable(13)
sig = Variable(2)

# Define objective function
objective = Minimize(c.T*s)

# Define constraints
# Apparent Power Limits
constraints = [s <= s_max]

# Nodal voltage limits
constraints += [v_min**2 <= V,
                V <= v_max**2]

# Squared line current limits
constraints += [L[0,0] <= I_max**2]

# Second order cone constraint for solar generator on node 9
constraints += [a_bar.T * hstack((s[9],sig)) + norm(E.T * hstack((s[9],sig))) <= 0, 0 <= sig, sig <= 1]

# Boundary condition for power line flows
constraints += [P[0,0] == 0,
                Q[0,0] == 0]

# Boundary condition for squared line current
constraints += [L[0] == 0]

# Fix node 0 voltage to be 1 "per unit" (p.u.)
constraints += [V[0] == 1]

# Loop over each node
for jj in j_idx:
    
    # Parent node, i = \rho(j)
    ii = rho[jj]
    
    # Line Power Flows
    constraints += [P[ii,jj] == l_P[jj] - p[jj] + r[ii,jj]*L[ii,jj] + A[jj,:]*P[jj,:].T,
                    Q[ii,jj] == l_Q[jj] - q[jj] + x[ii,jj]*L[ii,jj] + A[jj,:]*Q[jj,:].T]

    # Nodal voltage
    constraints += [V[jj] == V[ii] + (r[ii,jj]**2 + x[ii,jj]**2)*L[ii,jj] - 2*(r[ii,jj]*P[ii,jj] + x[ii,jj]*Q[ii,jj])]
    
    # Squared current magnitude on lines
    constraints += [L[ii,jj] >= quad_over_lin(vstack((P[ii,jj],Q[ii,jj])),V[jj])]
    
    # Compute apparent power from active & reactive power
    constraints += [norm(vstack((p[jj],q[jj]))) <= s[jj]]
    

# Define problem and solve
prob2 = Problem(objective, constraints)
prob2.solve()

# Output Results
print prob.status
print "Minimum Generating Cost : %4.2f"%(prob2.value),"USD"
print " "
print "Node 0 Nuclear Power: active = %1.3f"%(p[0].value), "MW | reactive = %1.3f"%(q[0].value), "MW | apparent = %1.3f"%(s[0].value),"MW || marginal savings = %3.0f"%(constraints[0].dual_value[0]), "USD/MW"
print "Node 3 FF Power: active = %1.3f"%(p[3].value), "MW | reactive = %1.3f"%(q[3].value), "MW | apparent = %1.3f"%(s[3].value),"MW || marginal savings = %3.0f"%(constraints[0].dual_value[3]), "USD/MW"
print "Node 9 Solar Power: active = %1.3f"%(p[9].value), "MW | reactive = %1.3f"%(q[9].value), "MW | apparent = %1.3f"%(s[9].value),"MW || marginal savings = %3.0f"%(constraints[0].dual_value[9]), "USD/MW"
print " "
print "Total active power   : %1.3f"%(np.sum(l_P)),"MW   demanded | %1.3f"%(np.sum(p.value)),"MW   generated"
print "Total reactive power : %1.3f"%(np.sum(l_Q)),"MVAr demanded | %1.3f"%(np.sum(q.value)),"MVAr generated"
print "Total apparent power : %1.3f"%(np.sum(l_S)),"MVA  demanded | %1.3f"%(np.sum(s.value)),"MVA  generated"
print " "
for jj in j_idx:
    print "Node %2.0f"%(jj), "Voltage : %1.3f"%((V[jj].value)**0.5), "p.u."

WARN: m less than n, problem likely degenerate
WARN: A->p (column pointers) not strictly increasing, column 105 empty
WARN: A->p (column pointers) not strictly increasing, column 106 empty
WARN: A->p (column pointers) not strictly increasing, column 107 empty
WARN: A->p (column pointers) not strictly increasing, column 108 empty
WARN: A->p (column pointers) not strictly increasing, column 109 empty
WARN: A->p (column pointers) not strictly increasing, column 110 empty
WARN: A->p (column pointers) not strictly increasing, column 111 empty
WARN: A->p (column pointers) not strictly increasing, column 112 empty
WARN: A->p (column pointers) not strictly increasing, column 113 empty
WARN: A->p (column pointers) not strictly increasing, column 114 empty
WARN: A->p (column pointers) not strictly increasing, column 115 empty
WARN: A->p (column pointers) not strictly increasing, column 116 empty
WARN: A->p (column pointers) not strictly increasing, column 118 empty
WARN: A->p (column pointers) n